<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90_%ED%86%B5%ED%95%A9%ED%85%8C%EC%9D%B4%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘 시작

## 라이브러리 설치

In [1]:
!pip install finance-datareader
!pip install pykrx
!pip install pymysql
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

## 패키지 & 함수 로딩

In [2]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

import datetime

In [3]:
def money_sur(stock_name, start, end):

    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [4]:
def reverse_min_max_scaling(org_x, x): #종가 예측값
    org_x_np = np.asarray(org_x) 
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min())) + org_x_np.min()

## 통합테이블 생성

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# total_table = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/total_table.csv') # 기존 테이블

In [7]:
# 구성
dic = {'code' : [],
       'date' : [],
       'lstm' : [],
       'arima' : [],
       'fbprophet' : [],
       'RL' : [],
    #    'mail_news' : [],
    #    'mail_news_nsi' : [],
    #    'asia_news' : [],
    #    'asia_news_nsi': [],
    #    'youtube_sam' : [],
    #    'youtube_su' : [],
    #    'youtube_han' : [],
       'close' : []}

total_table = pd.DataFrame(dic)

In [8]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'close'], dtype='object')

In [9]:
total_table

,code,date,lstm,arima,fbprophet,RL,close


## LSTM 예측결과

### 사용 데이터 로드 및 전처리

In [10]:
start_date = (datetime.datetime.now() - datetime.timedelta(30)).strftime("%Y%m%d")
end_date = datetime.datetime.now().strftime("%Y%m%d")
lstm = money_sur('005930', start_date, end_date)

In [11]:
lstm = lstm.set_index('Date')

In [12]:
lstm.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-09-01,76700,77100,75900,76800,16114775,0.001304,39406,19.99,1.95,3841,3.90,2994
2021-09-02,76800,76800,75700,76000,15347486,-0.010417,39406,19.79,1.93,3841,3.94,2994
2021-09-03,76400,76700,76000,76600,12096419,0.007895,39406,19.94,1.94,3841,3.91,2994
2021-09-06,76800,77600,76600,77300,12861180,0.009138,39406,20.12,1.96,3841,3.87,2994
2021-09-07,77100,77100,75900,76100,13239401,-0.015524,39406,19.81,1.93,3841,3.93,2994


In [13]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
lstm_sc = sc.fit_transform(lstm)

In [14]:
scale_cols = lstm.columns
lstm_sc_df = pd.DataFrame(lstm_sc, columns=[scale_cols], index=lstm.index)

In [15]:
lstm_sc_df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-09-01,0.736842,0.815789,0.707317,0.800000,0.507434,0.696202,0.0,0.800000,0.818182,0.0,0.200001,0.0
2021-09-02,0.763158,0.736842,0.658537,0.622222,0.459458,0.425448,0.0,0.633333,0.636363,0.0,0.360001,0.0
2021-09-03,0.657895,0.710526,0.731707,0.755556,0.256179,0.848459,0.0,0.758333,0.727273,0.0,0.240001,0.0
2021-09-06,0.763158,0.947368,0.878049,0.911111,0.303997,0.877189,0.0,0.908333,0.909091,0.0,0.080000,0.0
2021-09-07,0.842105,0.815789,0.707317,0.644444,0.327646,0.307465,0.0,0.650000,0.636363,0.0,0.320001,0.0


In [16]:
for i in scale_cols:
  for s in range(1, 11):  
    lstm_sc_df[(i+'shift_{}'.format(s),)] = lstm_sc_df[(i,)].shift(s)

In [17]:
lstm_sc_df.dropna()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,Openshift_1,Openshift_2,Openshift_3,Openshift_4,Openshift_5,Openshift_6,Openshift_7,Openshift_8,Openshift_9,Openshift_10,Highshift_1,Highshift_2,Highshift_3,Highshift_4,Highshift_5,Highshift_6,Highshift_7,Highshift_8,Highshift_9,Highshift_10,Lowshift_1,Lowshift_2,Lowshift_3,Lowshift_4,Lowshift_5,Lowshift_6,Lowshift_7,Lowshift_8,...,PBRshift_1,PBRshift_2,PBRshift_3,PBRshift_4,PBRshift_5,PBRshift_6,PBRshift_7,PBRshift_8,PBRshift_9,PBRshift_10,EPSshift_1,EPSshift_2,EPSshift_3,EPSshift_4,EPSshift_5,EPSshift_6,EPSshift_7,EPSshift_8,EPSshift_9,EPSshift_10,DIVshift_1,DIVshift_2,DIVshift_3,DIVshift_4,DIVshift_5,DIVshift_6,DIVshift_7,DIVshift_8,DIVshift_9,DIVshift_10,DPSshift_1,DPSshift_2,DPSshift_3,DPSshift_4,DPSshift_5,DPSshift_6,DPSshift_7,DPSshift_8,DPSshift_9,DPSshift_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-15,0.921053,0.894737,0.829268,0.844444,0.301993,0.786715,0.0,0.850000,0.818182,0.0,0.160001,0.0,0.842105,0.342105,0.368421,0.657895,0.552632,0.842105,0.763158,0.657895,0.763158,0.736842,0.973684,0.605263,0.421053,0.684211,0.631579,0.815789,0.947368,0.710526,0.736842,0.815789,0.878049,0.512195,0.439024,0.487805,0.634146,0.707317,0.878049,0.731707,...,0.727273,0.727273,0.454545,0.454545,0.727273,0.636363,0.909091,0.727273,0.636363,0.818182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240001,0.280001,0.520000,0.520000,0.280001,0.320001,0.080000,0.240001,0.360001,0.200001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-16,0.894737,0.894737,0.756098,0.644444,0.316906,0.396072,0.0,0.650000,0.636363,0.0,0.320001,0.0,0.921053,0.842105,0.342105,0.368421,0.657895,0.552632,0.842105,0.763158,0.657895,0.763158,0.894737,0.973684,0.605263,0.421053,0.684211,0.631579,0.815789,0.947368,0.710526,0.736842,0.829268,0.878049,0.512195,0.439024,0.487805,0.634146,0.707317,0.878049,...,0.818182,0.727273,0.727273,0.454545,0.454545,0.727273,0.636363,0.909091,0.727273,0.636363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.160001,0.240001,0.280001,0.520000,0.520000,0.280001,0.320001,0.080000,0.240001,0.360001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-17,0.631579,0.842105,0.707317,0.888889,0.518359,1.000000,0.0,0.891667,0.909091,0.0,0.120001,0.0,0.894737,0.921053,0.842105,0.342105,0.368421,0.657895,0.552632,0.842105,0.763158,0.657895,0.894737,0.894737,0.973684,0.605263,0.421053,0.684211,0.631579,0.815789,0.947368,0.710526,0.756098,0.829268,0.878049,0.512195,0.439024,0.487805,0.634146,0.707317,...,0.636363,0.818182,0.727273,0.727273,0.454545,0.454545,0.727273,0.636363,0.909091,0.727273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.320001,0.160001,0.240001,0.280001,0.520000,0.520000,0.280001,0.320001,0.080000,0.240001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-23,0.947368,0.947368,0.926829,0.933333,0.566260,0.725930,0.0,0.933333,0.909091,0.0,0.080000,0.0,0.631579,0.894737,0.921053,0.842105,0.342105,0.368421,0.657895,0.552632,0.842105,0.763158,0.842105,0.894737,0.894737,0.973684,0.605263,0.421053,0.684211,0.631579,0.815789,0.947368,0.707317,0.756098,0.829268,0.878049,0.512195,0.439024,0.487805,0.634146,...,0.909091,0.636363,0.818182,0.727273,0.727273,0.454545,0.454545,0.727273,0.636363,0.909091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.120001,0.320001,0.160001,0.240001,0.280001,0.520000,0.520000,0.280001,0.320001,0.080000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-24,0.973684,0.973684,1.000000,0.911111,0.265612,0.636237,0.0,0.908333,0.909091,0.0,0.080000,0.0,0.947368,0.631579,0.894737,0.921053,0.842105,0.342105,0.368421,0.657895,0.552632,0.842105,0.947368,0.842105,0.894737,0.894737,0.973684,0.605263,0.421053,0.684211,0.631579,0.815789,0.926829,0.707317,0.756098,0.829268,0.878049,0.512195,0.439024,0.487805,...,0.909091,0.909091,0.636363,0.818182,0.727273,0.727273,0.454545,0.454545,0.727273,0.636363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.080000,0.120001,0.320001,0.160001,0.240001,0.280001,0.520000,0.520000,0.280001,0.320001,0.0,0.0,0.0,0.0,0

In [18]:
X_test = lstm_sc_df.dropna().drop(columns=scale_cols, axis=1)
y_test = lstm_sc_df.dropna()[['Close']]

X_test= X_test.values
y_test = y_test.values

In [19]:
X_test

array([[0.84210526, 0.34210526, 0.36842105, ..., 0.        , 0.        ,
        0.        ],
       [0.92105263, 0.84210526, 0.34210526, ..., 0.        , 0.        ,
        0.        ],
       [0.89473684, 0.92105263, 0.84210526, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.89473684, 0.97368421, ..., 0.        , 0.        ,
        0.        ],
       [0.23684211, 1.        , 0.89473684, ..., 0.        , 0.        ,
        0.        ],
       [0.10526316, 0.23684211, 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
X_test_t = X_test.reshape(X_test.shape[0], 10, 12)

### 모델 로드

In [21]:
# 데이터 불러오기
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dense(32, activation = 'relu'))
model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'tanh'))
model.add(LSTM(64,
               return_sequences = False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1))



lstm_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/LSTM/'
samsung_lstm = 'lstm_samsung.h5'

model.load_weights(lstm_path+samsung_lstm)

### 모델 결과

In [23]:
pred = model.predict(X_test_t)

In [24]:
pred.shape, y_test.shape

((10, 1), (10, 1))

In [25]:
pred_act = reverse_min_max_scaling(lstm['Close'], pred)

In [26]:
total_table['date'] = lstm.index[-10:]
total_table['lstm'] = pred_act
total_table['close'] = lstm.Close[-10:].values.reshape(-1,1)
total_table['code'] = '005930'

In [27]:
total_table

,code,date,lstm,arima,fbprophet,RL,close
0,005930,2021-09-15,75726.278076,NaN,NaN,NaN,77000
1,005930,2021-09-16,75741.098633,NaN,NaN,NaN,76100
2,005930,2021-09-17,76177.318359,NaN,NaN,NaN,77200
3,005930,2021-09-23,76441.123779,NaN,NaN,NaN,77400
4,005930,2021-09-24,76502.779053,NaN,NaN,NaN,77300
5,005930,2021-09-27,76445.864990,NaN,NaN,NaN,77700
6,005930,2021-09-28,76496.509766,NaN,NaN,NaN,76300
7,005930,2021-09-29,76922.576172,NaN,NaN,NaN,74100
8,005930,2021-09-30,76402.852539,NaN,NaN,NaN,74100
9,005930,2021-10-01,75470.416016,NaN,NaN,NaN,73200


## arima 예측결과

In [28]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005930', '2018', end_date)

In [29]:
df = samsung_df[['Date','Close']]
df.columns = ['Date', 'Price']

In [30]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# 학습, 테스트 데이터 분리
train = df[:-1]
test = df[-1:]

model = ARIMA(train.Price.values, order = (2,1,2))
model_fit = model.fit(trend = 'nc', full_output = True, disp = True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                  922
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -7622.588
Method:                       css-mle   S.D. of innovations            940.031
Date:                Fri, 01 Oct 2021   AIC                          15255.176
Time:                        02:43:47   BIC                          15279.309
Sample:                             1   HQIC                         15264.385
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1.D.y     -1.5978      0.010   -157.401      0.000      -1.618      -1.578
ar.L2.D.y     -0.9813      0.008   -122.487      0.000      -0.997      -0.966
ma.L1.D.y      1.5993      0.009    179.475      0.0

In [31]:
fore = model_fit.forecast(steps=1) # 오늘 예측

# 오늘 예측 데이터
pred_arima_y = fore[0].tolist()

# 실제 오늘 데이터 
test_y = test.Price.values

# # 신뢰구간
# # 예측 데이터 최소값
# pred_y_lower = []
# # 예측 데이터 최대값
# pred_y_upper = []

# for lower_upper in fore[2]:
#     lower = lower_upper[0]
#     upper = lower_upper[1]
#     pred_y_lower.append(lower)
#     pred_y_upper.append(upper)

In [32]:
total_table['arima'].iloc[-1:] = pred_arima_y

In [33]:
total_table

,code,date,lstm,arima,fbprophet,RL,close
0,005930,2021-09-15,75726.278076,NaN,NaN,NaN,77000
1,005930,2021-09-16,75741.098633,NaN,NaN,NaN,76100
2,005930,2021-09-17,76177.318359,NaN,NaN,NaN,77200
3,005930,2021-09-23,76441.123779,NaN,NaN,NaN,77400
4,005930,2021-09-24,76502.779053,NaN,NaN,NaN,77300
5,005930,2021-09-27,76445.864990,NaN,NaN,NaN,77700
6,005930,2021-09-28,76496.509766,NaN,NaN,NaN,76300
7,005930,2021-09-29,76922.576172,NaN,NaN,NaN,74100
8,005930,2021-09-30,76402.852539,NaN,NaN,NaN,74100
9,005930,2021-10-01,75470.416016,74356.543739,NaN,NaN,73200


## fbprophet 예측

In [34]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005930', start = '20180101', end = end_date)
df1 = samsung_df[['Date','Close']]
df1.columns = ['ds', 'y']

from fbprophet import Prophet

prophet = Prophet(seasonality_mode = 'multiplicative',
                 yearly_seasonality=True, 
                 weekly_seasonality=True,
                 daily_seasonality=True,
                 changepoint_prior_scale=0.6)

prophet.fit(df1)

# 10일단위로 예측값을 가져옴
future_data = prophet.make_future_dataframe(periods = 10, freq = 'd')
forecast_data = prophet.predict(future_data)

In [35]:
list(total_table.date.values)

[numpy.datetime64('2021-09-15T00:00:00.000000000'),
 numpy.datetime64('2021-09-16T00:00:00.000000000'),
 numpy.datetime64('2021-09-17T00:00:00.000000000'),
 numpy.datetime64('2021-09-23T00:00:00.000000000'),
 numpy.datetime64('2021-09-24T00:00:00.000000000'),
 numpy.datetime64('2021-09-27T00:00:00.000000000'),
 numpy.datetime64('2021-09-28T00:00:00.000000000'),
 numpy.datetime64('2021-09-29T00:00:00.000000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000000'),
 numpy.datetime64('2021-10-01T00:00:00.000000000')]

In [36]:
total_table['fbprophet'] = forecast_data[forecast_data['ds'].isin(list(total_table.date.values))]['yhat'].values

In [37]:
total_table

,code,date,lstm,arima,fbprophet,RL,close
0,005930,2021-09-15,75726.278076,NaN,76623.235862,NaN,77000
1,005930,2021-09-16,75741.098633,NaN,76644.816688,NaN,76100
2,005930,2021-09-17,76177.318359,NaN,76621.520125,NaN,77200
3,005930,2021-09-23,76441.123779,NaN,76429.803592,NaN,77400
4,005930,2021-09-24,76502.779053,NaN,76287.749442,NaN,77300
5,005930,2021-09-27,76445.864990,NaN,75936.439522,NaN,77700
6,005930,2021-09-28,76496.509766,NaN,75967.821539,NaN,76300
7,005930,2021-09-29,76922.576172,NaN,75715.302244,NaN,74100
8,005930,2021-09-30,76402.852539,NaN,75549.748777,NaN,74100
9,005930,2021-10-01,75470.416016,74356.543739,75355.395550,NaN,73200


In [38]:
# total_table.to_csv('./total_table.csv', index =False)

## RL 결과

### 데이터 로딩 및 함수 로딩

In [39]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

# 금융정보 조회하기
def dataset_loaderKR(stock_name, start, end, train_ratio=0.8):
    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')
    date_split = data.index[int(train_ratio*len(data))] 

    return data[:date_split], data[date_split:], date_split


# ENV 설정
class Environment1:
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs

    def step(self, act):
        reward = 0
        
        # action
        # 0: Idle
        # 1: 매수
        # 2: 매도
        if act == 1: #매수
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # 매도
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

# MODEL
class Q_Network(nn.Module):
    def __init__(self,obs_len,hidden_size,actions_n):
        super(Q_Network,self).__init__()
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
    def forward(self,x):
        h = self.fc_val(x)
        return (h)

In [40]:
(train, test, date_split) = dataset_loaderKR('005930', start = '20180101', end = end_date)

### 모델 로딩 및 결과

In [41]:
rl_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/RL/삼성_결과/'
samsung_rl = 'samsung_rl_재무.h5'

In [42]:
# 모델 로딩
model = torch.load(rl_path+samsung_rl)

test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs

# sell remaining stocks
profits = 0
for p in test_env.positions:
	profits += (test_env.data.iloc[test_env.t, :]['Close'] - p)

test_env.profits += profits
test_env.positions = []

test_profits = test_env.profits
pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
pact = np.argmax(pact.data)
next_action = pact.item()
test['Action'] = test_acts + [next_action]


print( )
print( test )
print( )
print("결과")
print( "손익결과 $%.2f" % round(test_profits,2) )
# 0: Idle
# 1: 매수
# 2: 매도


          Date   Open   High    Low  Close  ...   PBR   EPS   DIV   DPS  Action
739 2021-01-05  81600  83900  81600  83900  ...  2.24  3166  1.69  1416       1
740 2021-01-06  83300  84500  82100  82200  ...  2.19  3166  1.72  1416       2
741 2021-01-07  82800  84200  82700  82900  ...  2.21  3166  1.71  1416       0
742 2021-01-08  83300  90000  83000  88800  ...  2.37  3166  1.59  1416       0
743 2021-01-11  90000  96800  89500  91000  ...  2.42  3166  1.56  1416       2
..         ...    ...    ...    ...    ...  ...   ...   ...   ...   ...     ...
919 2021-09-27  77300  77700  77000  77700  ...  1.97  3841  3.85  2994       2
920 2021-09-28  77700  77800  76200  76300  ...  1.94  3841  3.92  2994       0
921 2021-09-29  74800  75300  73800  74100  ...  1.88  3841  4.04  2994       1
922 2021-09-30  74300  74800  73700  74100  ...  1.88  3841  4.04  2994       2
923 2021-10-01  73900  74000  73000  73100  ...  1.86  3841  4.09  2994       2

[185 rows x 14 columns]

결과
손익결과 $-100

In [43]:
test['Action'] = np.where(test['Action'] == 1,1,
                          np.where(test['Action'] == 2,-1,0))

In [44]:
test['Action'][-10:]

914   -1
915    0
916   -1
917   -1
918   -1
919   -1
920    0
921    1
922   -1
923   -1
Name: Action, dtype: int64

In [45]:
total_table['RL'] = test[test['Date'].isin(list(total_table.date.values))]['Action'].values

In [46]:
total_table

,code,date,lstm,arima,fbprophet,RL,close
0,005930,2021-09-15,75726.278076,NaN,76623.235862,-1,77000
1,005930,2021-09-16,75741.098633,NaN,76644.816688,0,76100
2,005930,2021-09-17,76177.318359,NaN,76621.520125,-1,77200
3,005930,2021-09-23,76441.123779,NaN,76429.803592,-1,77400
4,005930,2021-09-24,76502.779053,NaN,76287.749442,-1,77300
5,005930,2021-09-27,76445.864990,NaN,75936.439522,-1,77700
6,005930,2021-09-28,76496.509766,NaN,75967.821539,0,76300
7,005930,2021-09-29,76922.576172,NaN,75715.302244,1,74100
8,005930,2021-09-30,76402.852539,NaN,75549.748777,-1,74100
9,005930,2021-10-01,75470.416016,74356.543739,75355.395550,-1,73200


## News 긍부정 결과

In [47]:
# corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion'] 참고용
stock_num_list = ['005930', '005380', '051910', '000660', '068270']
news = ['maeil_news_craw', 'asia_news_craw']
start = (datetime.datetime.now() - datetime.timedelta(10)).strftime("%Y-%m-%d").replace('-','')
end = datetime.datetime.now().strftime("%Y-%m-%d").replace('-', '')

In [48]:
# news_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/Final Data/1. samsung_score.csv') # 연습용

### 관련 함수 및 패키지 로드

In [49]:
import pymysql
import sys
import re

In [50]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

In [51]:
def news_db(news, stock):
    db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

    cursor = db.cursor(pymysql.cursors.DictCursor)
    news1 = news + '_news_craw'

    sql = "select * from {0}_{1} where (length(date)=10) and (date between {2}00 and {3}23)".format(news1, stock, start, end)
    cursor.execute(sql)
    result = cursor.fetchall()

    # DataFrame으로 변경
    df = pd.DataFrame(result)

    db.close()

    df.rename(columns={'date': 'datetime'}, inplace=True)

    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df['date'] = df['datetime'].str[0:4] + '-' + df['datetime'].str[4:6] + '-' + df['datetime'].str[6:8]
    df['date'] = pd.to_datetime(df['date'])

    # 결측치 제거 → 데이터 로드 시 완료했기 때문에 그다지 필요하지 않은 과정
    df = df.dropna()

    # 시간순으로 정렬
    df.sort_values('datetime', inplace=True)
    df.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)

    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])

    # Stop Words List에 각 매체명 추가
    except_media_list = ['매일경제', '매일', '경제', 'maeil', 'MK', 'mk',
                         '아시아경제', '아시아', 'Asia', 'ASIA', 'asia',
                         '삼프로TV', '삼프로', 'TV',
                         '슈카월드', '슈카', '월드'
                         '한국경제TV', '한국']
    for word in except_media_list:
        stopwords.append(word)
    
    #### 2. Preprocessing
    '''감성 어휘 사전 : negative / positive
       뉴스 데이터 : samsung / hyundai / lg / sk
       주식 데이터 : stock_samsung / stock_hyundai / stock_lg / stock_sk
       공휴일 데이터 : holidays'''
    
    
    ### 1) 뉴스 데이터 날짜 조정
    
    ## 1-1)업로드 시각 컬럼 추가
    df['time'] = df['datetime'].str[-2:]
    
    ## 1-2) 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']

    for j in range(len(df['time'])):
        if df['time'][j] in after_market:
            df['date'][j] += datetime.timedelta(1)
        else:
            pass
    
    ## 1-3) 텍스트 전처리
    # \n, \t, \r 제거
    df['text'] = df['text'].str.replace('[\n|\t|\r]', '')
    df['text'] = df['text'].str.replace('\[|\]', '')
    df['text'] = df['text'].str.replace('\<.*\>', '')
    df['text'] = df['text'].str.replace('\\', '')
    df['text'] = df['text'].str.replace('이미지', '')


    ### 2) 주말 및 공휴일 제외
    
    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df[df['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df[df['date'].isin(market_closed_list)]['date'].index:
            df['date'][hoil] += datetime.timedelta(1)
    
    ### 3) Tokenization 컬럼 추가
    
    df['Tokenization'] = 0
    rows = df.shape[0]
    for row in range(rows):
        hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                                    # 정규 표현식 → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
        result = hangeul.sub('', df['text'][row])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                                               # 형태소 추출
        nouns = okt.nouns(df['text'][row])
        nouns = [x for x in nouns if len(x) > 1]                                  # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]                          # 불용어 제거
        
        corpus = " ".join(nouns)                                                  # List를 String으로 변환
        df['Tokenization'][row] = corpus


    # 점수 산출

    df['Positive_Score'] = 0
    df['Negative_Score'] = 0
    df['Ratio'] = 0.1
    df['Pred'] = 0
    df['NSI'] = 0.1
    
    for score in range(len(df)):
        pos_score = 0 ; neg_score = 0
        
        for token in range(len(df['Tokenization'][score].split())):
            if df['Tokenization'][score].split()[token] in positive:
                pos_score += 1
            elif df['Tokenization'][score].split()[token] in negative:
                neg_score += 1
            else:
                pass
        
        df['Positive_Score'][score] = pos_score
        df['Negative_Score'][score] = neg_score
        
        # 긍정과 부정의 비율
        if (pos_score==0) and (neg_score==0):
            df['Ratio'][score] = 0.5   # 둘 다 0일 경우에는 긍정으로 가정
        else:
            df['Ratio'][score] = pos_score / (pos_score + neg_score)
        
        # 예측 결과
        if df['Ratio'][score]>=0.5:
            df['Pred'][score] = 1
        else:
            df['Pred'][score] = -1
        
        # 뉴스심리지수(NSI) 계산
        if (pos_score==0) and (neg_score==0):
            df['NSI'][score] = 101
        else:
            df['NSI'][score] = (pos_score - neg_score) / (pos_score + neg_score) * 100 + 100
    
    return df

### 매일경제

In [52]:
# mail_df = news_df[(news_df['news'] == '매일경제') & (news_df['st_cd'] == 5930)]

In [53]:
mail_df = news_db('maeil', '005930')

In [54]:
mail_df.head()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
0,삼성전자,005930,매일경제,2021092107,"LG가 끝내 못다이룬 꿈, 삼성전자가 대신 해낼까…갤럭시Z 내년 롤러블 출시설 솔솔",http://news.mk.co.kr/newsRead.php?no=904422&ye...,삼성 롤러블폰 예상. 사진 출처 = 레츠고디지털 폴더블폰을 세계 최초로 상용화한 삼...,2021-09-23,07,삼성 롤러 블폰 예상 사진 출처 레츠고 디지털 더블 세계 최초 상용 롤러블폰 세계 ...,1,0,1.000000,1,200.000000
1,삼성전자,005930,매일경제,2021092109,카카오 김범수의 '삼월천하'…주가하락에 이재용 부회장 다시 최고부자 등극,http://news.mk.co.kr/newsRead.php?no=904621&ye...,김범수 카카오 의장 김범수 카카오 이사회 의장이 규제리스크로 인한 주가하락으로 한국...,2021-09-23,09,김범수 카카오 의장 김범수 카카오 이사회 의장 규제 리스크 주가 하락 한국인 최고 ...,4,2,0.666667,1,133.333333
2,삼성전자,005930,매일경제,2021092118,카카오 김범수 '韓 최고 부자' 석달 천하…이재용 부회장에 다시 밀려,http://news.mk.co.kr/newsRead.php?no=905191&ye...,김범수 카카오 이사회 의장이 규제리스크로 인한 주가하락으로 한국인 최고 부자 타이...,2021-09-23,18,김범수 카카오 이사회 의장 규제 리스크 주가 하락 한국인 최고 부자 타이틀 반면 이...,1,1,0.500000,1,100.000000
3,삼성전자,005930,매일경제,2021092215,"""삼성전자에 무슨 일이…"" 미국 소비자 충성도 10위권 밖으로 밀려났다",http://news.mk.co.kr/newsRead.php?no=906626&ye...,지난 15일 서울 서초구 삼성딜라이트샵에 전시된 갤럭시 폴드3 제품.사진제공=연합뉴...,2021-09-23,15,지난 서울 서초구 라이트 전시 갤럭시 폴드 제품 사진 제공 연합뉴스 미국 실시 소비...,1,0,1.000000,1,200.000000
4,삼성전자,005930,매일경제,2021092216,"삼성전자 '시험대' 올랐다…노조 ""매년 영업익 25% 성과급 달라""",http://news.mk.co.kr/newsRead.php?no=906775&ye...,삼성전자 노사가 다음달 5일 상견례를 열고 임금교섭 절차를 시작한다. 노조는 연봉...,2021-09-23,16,삼성 노사 상견례 임금 교섭 절차 시작 노조 연봉 일괄 인상 요구 예정 지난해 이재...,0,0,0.500000,1,101.000000


In [55]:
mail_ratio = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
mail_nsi = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','NSI']]

In [56]:
mail_ratio = mail_ratio.groupby(['date']).mean().reset_index()
mail_nsi = mail_nsi.groupby(['date']).mean().reset_index()

In [57]:
mail_ratio.columns = ['date', 'mail_news']
mail_nsi.columns = ['date', 'mail_news_nsi']

In [58]:
total_table = pd.merge(total_table, mail_ratio, on='date', how='left')
total_table = pd.merge(total_table, mail_nsi, on='date', how='left')

In [59]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'close']]

In [60]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,close
0,005930,2021-09-15,75726.278076,NaN,76623.235862,-1,NaN,NaN,77000
1,005930,2021-09-16,75741.098633,NaN,76644.816688,0,NaN,NaN,76100
2,005930,2021-09-17,76177.318359,NaN,76621.520125,-1,NaN,NaN,77200
3,005930,2021-09-23,76441.123779,NaN,76429.803592,-1,0.698718,140.128205,77400
4,005930,2021-09-24,76502.779053,NaN,76287.749442,-1,0.438889,87.944444,77300
5,005930,2021-09-27,76445.864990,NaN,75936.439522,-1,0.465548,93.382395,77700
6,005930,2021-09-28,76496.509766,NaN,75967.821539,0,0.571807,114.679654,76300
7,005930,2021-09-29,76922.576172,NaN,75715.302244,1,0.509740,102.038961,74100
8,005930,2021-09-30,76402.852539,NaN,75549.748777,-1,0.508242,101.956044,74100
9,005930,2021-10-01,75470.416016,74356.543739,75355.395550,-1,NaN,NaN,73200


### 아시아

In [61]:
# asia_df = news_df[(news_df['news'] == '아시아경제') & (news_df['st_cd'] == 5930)]

In [62]:
asia_df = news_db('asia', '005930')

In [69]:
asia_df.tail()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
133,삼성전자,005930,아시아경제,2021092916,"고민정 ""'곽상도 아들로 못 태어난 게 죄'라는 청년들 허탈감에 귀 기울여야""",https://view.asiae.co.kr/article/2021092916011...,"""당당함에 놀랐다…50억, 저조차도 평생 만질 수 없을 거라고 장담""""화천대유는 곽...",2021-09-30,16,평생 장담 화천 대유 의원 우렁 각시 요술 램프 고민정 민주당 의원 지난 오후 경남...,0,2,0.000000,-1,0.000000
134,삼성전자,005930,아시아경제,2021092916,"고민정 ""'곽상도 아들로 못 태어난 게 죄'라는 청년들 허탈감에 귀 기울여야""",https://view.asiae.co.kr/article/2021092916011...,"""당당함에 놀랐다…50억, 저조차도 평생 만질 수 없을 거라고 장담""""화천대유는 곽...",2021-09-30,16,평생 장담 화천 대유 의원 우렁 각시 요술 램프 고민정 민주당 의원 지난 오후 경남...,0,2,0.000000,-1,0.000000
135,삼성전자,005930,아시아경제,2021092916,美 '증시 충격'에 코스피·코스닥 동반 하락 마감,https://view.asiae.co.kr/article/2021092916084...,"美증시 급락 여파… 안전자산 선호 심리 강화반도체 투심 위축, 삼전·SK하이닉스 3...",2021-09-30,16,증시 급락 여파 안전 자산 선호 심리 강화 반도체 투심 위축 삼전 하이닉스 내외 하...,3,63,0.045455,-1,9.090909
136,삼성전자,005930,아시아경제,2021092920,"'급식 몰아주기' 2300억 과징금 삼성, 공정위 상대 행정소송 제기",https://view.asiae.co.kr/article/2021092920041...,썝蹂몃낫湲 븘씠肄아시아경제 강나훔 기자 사내급식 일감을 계열사에 몰아준 혐의로 23...,2021-09-30,20,몃낫 븘씠 나훔 기자 사내 급식 일감 열사 혐의 과징금 부과 삼성 정거 위원회 상대...,0,0,0.500000,1,101.000000
137,삼성전자,005930,아시아경제,2021093008,"""없어서 못산다"" 삼성 폴더블 이어 출격 앞둔 ‘아이폰13’도 품귀 예고",https://view.asiae.co.kr/article/2021092908215...,썝蹂몃낫湲 븘씠肄아시아경제 조슬기나 기자 “없어서 못 산다.” 출시 한 달이 지나고...,2021-09-30,08,몃낫 븘씠 슬기 기자 산다 출시 달이 서도 삼성 더블 갤럭시 플립 출격 아이폰 감치...,2,3,0.400000,-1,80.000000


In [64]:
asia_ratio = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
asia_nsi = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','NSI']]

asia_ratio = asia_ratio.groupby(['date']).mean().reset_index()
asia_nsi = asia_nsi.groupby(['date']).mean().reset_index()

asia_ratio.columns = ['date', 'asia_news']
asia_nsi.columns = ['date', 'asia_news_nsi']

In [65]:
total_table = pd.merge(total_table, asia_ratio, on='date', how='left')
total_table = pd.merge(total_table, asia_nsi, on='date', how='left')

In [66]:
# 컬럼 리셋
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close']]

In [67]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close
0,005930,2021-09-15,75726.278076,NaN,76623.235862,-1,NaN,NaN,NaN,NaN,77000
1,005930,2021-09-16,75741.098633,NaN,76644.816688,0,NaN,NaN,NaN,NaN,76100
2,005930,2021-09-17,76177.318359,NaN,76621.520125,-1,NaN,NaN,NaN,NaN,77200
3,005930,2021-09-23,76441.123779,NaN,76429.803592,-1,0.698718,140.128205,0.537567,107.680104,77400
4,005930,2021-09-24,76502.779053,NaN,76287.749442,-1,0.438889,87.944444,0.645292,129.058442,77300
5,005930,2021-09-27,76445.864990,NaN,75936.439522,-1,0.465548,93.382395,0.683408,136.865716,77700
6,005930,2021-09-28,76496.509766,NaN,75967.821539,0,0.571807,114.679654,0.247715,49.543050,76300
7,005930,2021-09-29,76922.576172,NaN,75715.302244,1,0.509740,102.038961,0.454242,91.040657,74100
8,005930,2021-09-30,76402.852539,NaN,75549.748777,-1,0.508242,101.956044,0.282424,56.684848,74100
9,005930,2021-10-01,75470.416016,74356.543739,75355.395550,-1,NaN,NaN,NaN,NaN,73200


In [68]:
# total_table.to_csv('total_table.csv', index=False)

## youtube 긍부정 결과

In [51]:
# # 이전 라벨링 데이터
# youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_naver_labeling.csv')

In [52]:
youtube_df.ch_nm.unique()

array(['한국경제TV', '슈카월드', '삼프로TV_경제의신과함께'], dtype=object)

### youtube 데이터 전체 로드

In [ ]:
def youtube_db():

    com_list = {'하이닉스' : '000660',
            '현대차' : '005380',
            '삼성전자' : '005930',
            'LG화학' : '051910',
            '셀트리온' : '068270'}
    media_list = {'youtube_한국경제_':'hk_',
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}
    df_list = []
    for n,m in media_list.items():
        for i,j in com_list.items():

            db = pymysql.connect(
                user='root', 
                passwd='1234', 
                host='3.35.70.166', 
                db='proj', 
                charset = 'utf8'
            )

            cursor = db.cursor(pymysql.cursors.DictCursor)
            sql = "select * from youtube_"+ m + j +" where length(date) = 10"
            
            cursor.execute(sql)

            result = cursor.fetchall()

            # 데이터 프레임으로 변경
            globals() ['df_'+ m + j ] = pd.DataFrame(result)
            print('df_'+ m + j)
            df_list.append(globals() ['df_'+ m + j ])
            
            db.close()
            
            df_total = pd.concat(df_list)
            
    return df_total

In [ ]:
df_total = youtube_db()

### 라벨링 패키지 및 모델 로딩

In [62]:
!pip konlpy
from konlpy.tag import Okt
okt = Okt()

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
tokenizer = Tokenizer()
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

ERROR: unknown command "konlpy"


In [64]:
max_len = 30

In [59]:
loaded_model = load_model('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/best_model.h5')

In [65]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    if(score > 0.5):
        # print("{:.2f}% 확률로 긍정 리뷰\n".format(score * 100))
        return score, 1
    elif (score == 0.5):
        return score, 0
    else:
        # print("{:.2f}% 확률로 부정 리뷰\n".format((1 - score) * 100))
        return (1 - score), -1

### 삼프로

In [ ]:
sampro_df = youtube_df[(youtube_df.ch_nm == '삼프로TV_경제의신과함께') & (youtube_df.st_cd == 5930)]

In [ ]:
sampro_df

,st_n,st_cd,ch_nm,date,title,text,views
1470,삼성전자,5930,삼프로TV_경제의신과함께,2019-08-26,재무제표도 모르면서 투자를 한다고?! I 재무제표를 읽어라 4부작 - 1부,박미령 말씀대로 못했어요경제 신들과 함께 하는 고품격 경제 파괴 신과함께초심자 특히...,794464
1508,삼성전자,5930,삼프로TV_경제의신과함께,2019-09-20,"전술적 자산배분의 기본 원칙_19.09.19_영주 닐슨, 박제영 [퇴근길 Page2]",4 오늘 하루도 수고하셨습니다3% 티비 저녁 라이브 퇴근길 페이지 투의 진행을 맡은...,13934
1536,삼성전자,5930,삼프로TV_경제의신과함께,2019-10-07,KT&G를 통해서 보는 잃지 않는 종목 고르는 팁! [최준철 대표의 주식 FM가이드...,으철 주철 대표와 함께하는 주식 4 fm 카이드 다섯번째 시간이시작되었습니다반갑습니...,49004
1559,삼성전자,5930,삼프로TV_경제의신과함께,2019-10-22,"왜 삼성전자만 오르고 하이닉스는 쉬지?_19.10.22_김영우,곽상준 [오늘아침 P...",[음악]으으깊이가 다른 경제 컨텐츠 오늘아침 페이지 투 정영진김도원 입니다[음악]내...,13434
1576,삼성전자,5930,삼프로TV_경제의신과함께,2019-10-29,"삼성전자, 인텔과 반도체 치킨 게임 시작하나?_19.10.29_이주완,곽상준 [오늘...",[음악]4으깊이가 다른 경제 컨텐츠 오나지 페이지 투정유진 김도원 입니다[음악]슈얼...,14357
...,...,...,...,...,...,...,...
6350,삼성전자,5930,삼프로TV_경제의신과함께,2021-09-30,"[퇴근길 라이브&백브리핑] 퀀트전략, 올해보다 내년이 더 좋을 기업 / 전세대출 규...",[음악]으으 으oo 아아 으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는 ...,201424
6351,삼성전자,5930,삼프로TV_경제의신과함께,2021-09-30,[글로벌 개장시황] 불확실의 해소? 8월 보다 공포지수 줄어든 미국증시 f. 미국주...,블롭 얼라이브 3부 시작하도록하겠습니다어제는 빨리 2분 봐야되는데 하고있었습니다 장...,23267
6352,삼성전자,5930,삼프로TV_경제의신과함께,2021-09-30,[주인장백 브리핑] KB 이어 하나은행도 전세대출 한도 줄인다 / 몸값 많이 올라간...,4암튼 회장님의 어느 또 재미있는 소식갖고 오셨어요 예 그 재생 플라스틱관련해서먼지...,12019
6353,삼성전자,5930,삼프로TV_경제의신과함께,2021-09-30,[글로벌 이슈체크] 내년부터 메타버스 본게임?! 기업들의 전망은? f. 미래에셋증권...,디몰 iv 부 시작하도록 하겠습니다요즘은 장 방송 잠깐 중간에 게스트분하고 인상안 ...,15399


### 매일경제

### 슈카월드